In [ ]:
import glob
import tensorflow
import math
import os
import random
import json
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from datetime import datetime as dt
from tensorflow.keras.models import model_from_json
from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger, LearningRateScheduler
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
#from losses import categorical_focal_loss

from tensorflow.keras.applications.densenet import preprocess_input
import tensorflow as tf
from tensorflow.keras.preprocessing import image
print("TF Version:", tf.__version__)

In [ ]:
train_dir = '/Users/aravindagayan/Documents/Projects/DataSets/Cat_Dog_data/train'
test_dir = '/Users/aravindagayan/Documents/Projects/DataSets/Cat_Dog_data/test'

#model_json = 
#model_weight = 

MODEL_NAME = 'cats_dogs_classifier'
BATCH_SIZE = 1
EPOCHS = 1
IMG_HEIGHT = 224
IMG_WIDTH = 224
lr=0.001
PRETRAINED_MODEL = 'densenet201'
CHECKPOINT_PATH = './models'
TENSORBOARD_PATH = './tensorboard'

### Check Images

In [ ]:
import pygal 
from IPython.display import display, HTML

#Create function to display interactive plotting
base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

def galplot(chart):
    rendered_chart = chart.render(is_unicode=True)
    plot_html = base_html.format(rendered_chart=rendered_chart)
    display(HTML(plot_html))
    
#Compare class distribution
line_chart = pygal.Bar(height=300)
line_chart.title = 'Class Distribution'
for o in os.listdir(train_dir):
    if not o.startswith('.'):
        line_chart.add(o, len(os.listdir(os.path.join(train_dir, o))))
galplot(line_chart)

In [ ]:
file_list = glob.glob(os.path.join(train_dir,'*','*'))
img_path = random.choice(file_list)
img = load_img(img_path)
car_class = img_path.split("/")[-2]
plt.imshow(img)
plt.axis("off")
plt.title(car_class, fontsize=16)

### Checking for corrupted Files

In [ ]:
from os import listdir
from PIL import Image

file_list = [i for i in glob.glob(os.path.join(train_dir,'*','*')) if i.split('/')[-1].split('.')[-1].lower() in ['jpg','png','jpeg']]
   
for filename in file_list:
    try:
        img = Image.open(filename) # open the image file
        img.verify() # verify that it is, in fact an image
    except (IOError, SyntaxError) as e:
        print('Bad file: ', filename) # print out the names of corrupt files
print('Checked '+ str(len(file_list))+' Images')

### Seperate train, val, test set

In [ ]:
import math
import re
import sys
import shutil

#Train and Test Set Variables
orginal_data_path ='/Users/aravindagayan/Documents/Projects/DataSets/TR'

train_val_test_ratio = (.8, .1, .1) #  Data Split
test_folder = '/Users/aravindagayan/Documents/Projects/DataSets/Cat_Dog_data/test/'
train_folder = '/Users/aravindagayan/Documents/Projects/DataSets/Cat_Dog_data/train/'
val_folder = '/Users/aravindagayan/Documents/Projects/DataSets/Cat_Dog_data/val/'

file_names = os.listdir(orginal_data_path)
file_names = [i for i in file_names if not i.startswith('.')]

#Remove Existing Folders if they exist
for folder in [test_folder, train_folder, val_folder]:
    if os.path.exists(folder) and os.path.isdir(folder):
        shutil.rmtree(folder)
        
#Remake Category Folders in both Train and Test Folders
for category in file_names:
    os.makedirs(test_folder + category)
    os.makedirs(train_folder + category)
    os.makedirs(val_folder + category)

#Split Data by Train Ratio and copy files to correct directory
for idx, category in enumerate(file_names):
    file_list = os.listdir(orginal_data_path + '/' + category)
    
    train_ratio = math.floor(len(file_list) * train_val_test_ratio[0])
    val_ratio = math.floor(len(file_list) * train_val_test_ratio[1])
    train_list = file_list[:train_ratio]
    val_list = file_list[train_ratio:train_ratio + val_ratio]
    test_list = file_list[train_ratio + val_ratio:]
    
    for i, file in enumerate(train_list):
        shutil.copy(orginal_data_path + '/' + category + '/' + file, train_folder + '/' + category + '/' + file)
    sys.stdout.write('Moving %s train images to category folder %s' % (len(train_list), category))  
    sys.stdout.write('\n')
    for i, file in enumerate(val_list):
        shutil.copy(orginal_data_path + '/' + category + '/' + file, val_folder + '/' + category + '/' + file)
    sys.stdout.write('Moving %s validation images to category folder %s' % (len(val_list), category))                   
    sys.stdout.write('\n')
    for i, file in enumerate(test_list):
        shutil.copy(orginal_data_path + '/' + category + '/' + file, test_folder + '/' + category + '/' + file)
    sys.stdout.write('Moving %s test images to category folder %s' % (len(test_list), category))
    sys.stdout.write('\n')
print("Done.")  

### Image Iterator

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.1)

train_flow = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size = BATCH_SIZE,
    subset='training',
    shuffle=True)

val_flow = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size = BATCH_SIZE,
    subset='validation',
    shuffle=False)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_flow = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size = BATCH_SIZE,
    shuffle=False)

### Check Train set images

In [ ]:
CLASS_NAMES = [k for k,v in train_flow.class_indices.items()]
CLASS_NAMES

In [ ]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        plt.title(CLASS_NAMES[label_batch[n].argmax()].title())
        plt.axis('off')

In [ ]:
image_batch, label_batch = next(train_flow)
show_batch(image_batch, label_batch)

### Download pretrained models

In [ ]:
#pretrained = tensorflow.keras.applications.densenet.DenseNet201(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling='max')
#pretrained.save('densenet201_pretrained_pool_max.h5')

In [ ]:
inputs = Input((IMG_HEIGHT, IMG_WIDTH, 3))
base_model = tf.keras.models.load_model('./models/pretrained_models/densenet201_pretrained_pool_max.h5')

x = Dense(units=1024, activation='relu')(base_model(inputs))
x = Dropout(rate=.4)(x)
outputs = Dense(2, name='output', activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(lr, epsilon=1e-08)

for layer in model.layers[:2]:
    layer.trainable = False

#for layer in model.layers:
#    layer.trainable = True
    
#model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(loss=categorical_focal_loss(alpha=.25, gamma=2), metrics=["accuracy"], optimizer=optimizer)
model.summary()

print('Trainable Layers:/n')
for layer in model.layers:
    print(layer.name, ':', layer.trainable)

In [ ]:
# Save model config to JSON
model_json = model.to_json()
with open(os.path.join('./models', '{}_{}_{}_{dt}.json'.format(MODEL_NAME,
                                                                      PRETRAINED_MODEL,
                                                                      lr,
                                                                      dt=dt.now().strftime("%Y-%m-%d--%H-%M"))), 'w') as json_file:
    json.dump(model_json, json_file)


In [ ]:
# Model checkpoint
model_checkpoint = ModelCheckpoint(filepath=os.path.join(CHECKPOINT_PATH, '{}_{}_{}_Epoch{{epoch:02d}}_{dt}.hdf5'.format(MODEL_NAME,
                                                                                                        PRETRAINED_MODEL,
                                                                                                        lr,
                                                                                                        dt=dt.now().strftime("%Y-%m-%d--%H-%M"))),
                                                                                                        monitor='val_loss',
                                                                                                        verbose=1, save_best_only=True, save_weights_only=True)
# Tensorboard
tensorboard = TensorBoard(log_dir=TENSORBOARD_PATH + 'tb_{}_{}_{}_{dt}/'.format(MODEL_NAME, PRETRAINED_MODEL,
                                                                                lr,
                                                                                dt=dt.now().strftime("%Y-%m-%d--%H-%M")),
                                                                                histogram_freq=0,
                                                                                write_graph=True,
                                                                                write_images=False)

# Logger
csv_logger = CSVLogger(TENSORBOARD_PATH + 'TrainLog_{}_{}_{}_{dt}.csv'.format(MODEL_NAME,
                                                                                PRETRAINED_MODEL,
                                                                                lr,
                                                                                dt=dt.now().strftime("%Y-%m-%d--%H-%M"),
                                                                                separator=';',
                                                                                append=True))
# Callbacks
callbacks_list = [model_checkpoint, tensorboard, csv_logger]

In [ ]:
model.fit(train_flow,
          epochs=EPOCHS,
          verbose=1,
          validation_data=val_flow,
          validation_steps=math.ceil(val_flow.samples / val_flow.batch_size),
          steps_per_epoch=math.ceil(train_flow.samples / train_flow.batch_size),
          callbacks=callbacks_list)

In [ ]:
with open(model_json, 'r') as f:
    model = tf.keras.models.model_from_json(json.loads(f.read()))
model.load_weights(model_weight)

In [ ]:
file_list = glob.glob(os.path.join(test_dir,'*',"*"))
img_path = random.choice(file_list)
img_cat = os.path.split(os.path.dirname(img_path))[1]
print("Image Name: ", img_path)
img = image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
plt.imshow(img)
plt.axis("off")
plt.title(img_cat, fontsize=16)

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
print("Predictions: ", preds)
print(preds[0].argmax())
test_flow.class_indices

label = [k for k,v in test_flow.class_indices.items()]
top_x = 3
top_args = preds[0].argsort()[-top_x:][::-1]
preds_label = [label[p] for p in top_args]
print("\nTop " + str(top_x) + " confidence: " + " ".join(map(str, sorted(preds[0])[-top_x:][::-1])))
print("Top " + str(top_x) + " labels: " + " ".join(map(str, preds_label)))